<a href="https://colab.research.google.com/github/pedrogengo/DLforNLP/blob/main/Aula_5_Exerc%C3%ADcio_Pedro_Gengo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: Pedro Gabriel Gengo Lourenço

In [ ]:
!pip install neptune-client

     |████████████████████████████████| 268 kB 8.1 MB/s 
     |████████████████████████████████| 829 kB 29.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 170 kB 46.6 MB/s 
     |████████████████████████████████| 131 kB 42.8 MB/s 
     |████████████████████████████████| 7.9 MB 66.0 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 138 kB 51.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 127 kB 52.6 MB/s 
     |████████████████████████████████| 129 kB 51.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.10.10-py2.py3-none-any.whl size=464917 sha256=dc683de15272de2216de759ae42ff6d579dc3571dca2695245a8f841f050835c
  Stored in directory: /root/.cache/pip/wheels/90/41/d8/7b2abc93762861e492f7fd68b7769b2b130501b46e7775b5b6
  Created wheel for future: filen

# Fixando a seed

In [ ]:
import random
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict

import neptune.new as neptune

In [ ]:
def set_seed():
  random.seed(123)
  np.random.seed(123)
  torch.manual_seed(123)
  torch.cuda.manual_seed(123)
set_seed()

## Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

--2021-09-15 14:47:35--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 172.67.69.159, 104.26.2.19, 104.26.3.19, ...
Connecting to files.fast.ai (files.fast.ai)|172.67.69.159|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/aclImdb.tgz [following]
--2021-09-15 14:47:35--  https://files.fast.ai/data/aclImdb.tgz
Connecting to files.fast.ai (files.fast.ai)|172.67.69.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [application/x-gtar-compressed]
Saving to: ‘aclImdb.tgz’

aclImdb.tgz         100%[===================>] 139.22M  63.5MB/s    in 2.2s    

2021-09-15 14:47:37 (63.5 MB/s) - ‘aclImdb.tgz’ saved [145982645/145982645]



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
import os
import random


def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
y_valid = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
False This film is terrible. I was really looking forward to it, as I thought "Lantana" was great.<br /><b
False I live in Ottawa where this film was made and I really wish it hadn't been. This is one God-awful fl
True This is an excellent, fast paced thriller by Wes Craven (Nightmare on Elm Street), who for 85 minute
3 últimas amostras treino:
False The Good:<br /><br />Effective color scheme. Good costumes. Top notch set production. Well detailed 
True Gregory Peck gives a brilliant performance in this film. The last 15 minutes (or thereabouts) are gr
True Michael Caton-Jones's Scottish period piece bears little connection to the Sir Walter Scott novel of
3 primeiras amostras validação:
True Hardly a masterpiece. Not so well written. Beautiful cinematography i think not. This movie wasn't t
True To get in touch with the beauty of this film pay close attention to the sound

In [ ]:
sum([len(item.split()) for item in x_train])

4674365

# Tokenização

Nessa etapa, o objetivo é pegar os textos de treino, validação e teste e tokenizá-los, ou seja, receber um _string_ e transformá-la em uma lista de tokens/palavras.

Para garantir o uso de sequências de mesmo tamanho, também aplicaremos um filtro para termos no máximo 200 tokens em cada texto tokenizado.

In [ ]:
from re import findall

def tokenizer(texts, size):
  tokenized_texts = []
  for text in texts:
    tokens = findall(r'\w+|[^?\-!.,:;"\'/><\s]', text)
    tokenized = [token.lower() for token in tokens][:size]
    tokenized_texts.append(tokenized)
  return tokenized_texts

In [ ]:
SENTENCE_SIZE = 200
tokenized_x_train = tokenizer(x_train, SENTENCE_SIZE)
tokenized_x_valid = tokenizer(x_valid, SENTENCE_SIZE)
tokenized_x_test = tokenizer(x_test, SENTENCE_SIZE)

# Criação do vocabulário

Para garantir que tenhamos o mesmo vocabulário tanto para o experimento com Bag of Words quanto para o experimento com Embeddings, e com o intuito de evitar o reprocessamento, criamos um dataset que será utilizado em todos os experimentos.

Para isso, recebemos todos os textos tokenizados de treino e aplicamos o Counter em cima deles. Ao final, percorremos todos os elementos desse vocabulário associando cada token com sua posição baseada na quantidade de vezes que apareceu, gerando assim o índice que cada token terá.

In [ ]:
from collections import Counter
def create_vocab(tokenized_texts, max_size=None):
    counter = Counter()
    for text in tokenized_texts:
      counter.update(text)
    vocab = {element[0]: index for index, element in enumerate(counter.most_common(max_size))}
    return vocab

In [ ]:
VOCAB = create_vocab(tokenized_x_train)
VOCAB['__UNK__'] = len(VOCAB)

# Vetorizadores

Abaixo temos a definição da classe BagOfWords e EmbeddingVectorizer. Ambas serão utilizadas para aplicar a transformação de um dado, seja de treino ou não, em tempo de execução do treino.

No método _.transform()_ da classe BagOfWords, recebemos uma lista de textos tokenizados, percorremos cada um deles criando um novo vetor, do tamanho do vocabulário, preenchido por zeros. Aplicamos então um Counter nesse texto tokenizado e substituimos no vetor de zeros, nos índices correspondentes a cada uma das palavras encontrar no texto, pela contagem do token.

Já o método _.transform()_ , passamos também um conjunto de textos tokenizados e, para cada texto, convertemos cada token existente em um índice, utilizando o dicionário. Após isso, verificamos se o tamanho do vetor gerado é menor que 200 e, caso seja, inserimos tokens PAD (tamanho do vocab) até completar.

In [ ]:
class BagOfWords():
  def __init__(self, vocab):
    self.vocab = vocab
  
  def fit(self):
    pass
  
  def transform(self, texts):
    transformed_texts = []
    for i, text in enumerate(texts):
      bow_text = torch.zeros(len(self.vocab))
      unk = self.vocab['__UNK__']
      counter = Counter(text)
      for key, value in counter.items():
        # if key in self.vocab:
        bow_text[self.vocab.get(key, unk)] += value
      transformed_texts.append(bow_text)
    return torch.vstack(transformed_texts).float()

In [ ]:
class EmbeddingVectorizer():

  def __init__(self, vocab, sequence_size=200):
    self.vocab = vocab
    self.sequence_size = sequence_size
  
  def fit(self):
    pass
  
  def transform(self, texts):
    transformed_texts = []
    for i, text in enumerate(texts):
      unk = self.vocab['__UNK__']
      emb_text = []
      for key in text:
        # if key in self.vocab:
        emb_text.append(self.vocab.get(key, unk))
      if len(emb_text) < 200:
        emb_text = emb_text + [len(self.vocab)] * (self.sequence_size - len(emb_text))
      transformed_texts.append(torch.LongTensor(emb_text))
    return torch.vstack(transformed_texts)

# Dataset e Dataloader

Na classe Dataset, a diferença é a aplicação do método _.transform()_ dentro do método *.\_\_getitem\_\_*. Com isso, evitamos sobrecarregar a memória com a base de dados inteira vetorizada e fazemos isso em tempo de execução do treinamento.

Já no DataLoader, englobei ele em uma função que gera as entradas da rede neural. Essa função monta cada um dos Datasets e depois monta os DataLoaders e os retorna.

In [ ]:
class Dataset(Dataset):

  def __init__(self, x, y, vectorizer=None):
    self.x = x
    self.y = y
    self.vectorizer = vectorizer
  
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    return self.vectorizer.transform([self.x[idx]])[0], self.y[idx]

In [ ]:
def create_nn_input(tokenized_text_train, y_train,
                    tokenized_text_valid, y_valid,
                    tokenized_text_test, y_test,
                    vectorizer, batch_size):
  

  train_dataset = Dataset(tokenized_text_train, torch.Tensor(y_train).reshape(-1,1), vectorizer)
  valid_dataset = Dataset(tokenized_text_valid, torch.Tensor(y_valid).reshape(-1,1), vectorizer)
  test_dataset = Dataset(tokenized_text_test, torch.Tensor(y_test).reshape(-1,1), vectorizer)

  vectorized_texts_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
  vectorized_texts_valid = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
  vectorized_texts_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return vectorized_texts_train, vectorized_texts_valid, vectorized_texts_test, vectorizer

# Loop de treinamento e validação

In [ ]:
run = neptune.init(
    project="pedro.gengo/IA-376",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjYyNDA1MS1hZDJlLTRiZDctYjIxNy0xMTNhY2FmNzZhYmIifQ==",
)

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/pedro.gengo/IA-376/e/IA-4
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
   print(torch. cuda. get_device_name(dev))
else: 
   dev = "cpu" 
print(dev)
device = torch.device(dev)

cpu


In [ ]:
def train(model, train, valid, criterion, optimizer, filename_save, n_epochs=10):
  
  best_valid_loss = 10e9
  best_epoch = 0
  train_losses, valid_losses = [], []
  for i in range(n_epochs):
    accumulated_loss = 0
    model.train()
    for x_train, y_train in train:
      x_train = x_train.to(device)
      y_train = y_train.to(device)
      outputs = model(x_train)
      batch_loss = criterion(outputs, y_train)

      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()
      accumulated_loss += batch_loss.item()

    train_loss = accumulated_loss / len(train.dataset)
    run[f"{filename_save}_train/loss"].log(train_loss)
    train_losses.append(train_loss)

    # Laço de Validação, um a cada época.
    accumulated_loss = 0
    accumulated_accuracy = 0
    model.eval()
    with torch.no_grad():
        for x_valid, y_valid in valid:
            x_valid = x_valid.to(device)
            y_valid = y_valid.to(device)

            # predict da rede
            outputs = model(x_valid)

            # calcula a perda
            batch_loss = criterion(outputs, y_valid)
            preds = outputs > 0.5

            # calcula a acurácia
            batch_accuracy = (preds == y_valid).sum()
            accumulated_loss += batch_loss
            accumulated_accuracy += batch_accuracy

    valid_loss = accumulated_loss / len(valid.dataset)
    run[f"{filename_save}_valid/loss"].log(valid_loss)
    valid_losses.append(valid_loss)

    valid_acc = accumulated_accuracy / len(valid.dataset)
    run[f"{filename_save}_valid/acc"].log(valid_acc)

    print(f'Época: {i:d}/{n_epochs - 1:d} Train Loss: {train_loss:.6f} Valid Loss: {valid_loss:.6f} Valid Acc: {valid_acc:.3f}')

    # Salvando o melhor modelo de acordo com a loss de validação
    if valid_loss < best_valid_loss:
        torch.save(model.state_dict(), filename_save + '.pt')
        best_valid_loss = valid_loss
        best_epoch = i
        print('best model')

  return model, train_losses, valid_losses

In [ ]:
def predict(model, state_dict, test):
  accumulated_accuracy = 0
  model.load_state_dict(torch.load(state_dict + '.pt'))
  model.eval()
  with torch.no_grad():
      for x_test, y_test in test:
          x_test = x_test.to(device)
          y_test = y_test.to(device)

          # predict da rede
          outputs = model(x_test)

          # calcula a perda
          batch_loss = criterion(outputs, y_test)
          preds = outputs > 0.5

          # calcula a acurácia
          batch_accuracy = (preds == y_test).sum()
          accumulated_accuracy += batch_accuracy

  test_acc = accumulated_accuracy / len(test.dataset)
  test_acc *= 100
  print('*' * 40)
  print(f'Acurácia de {test_acc:.3f} %')
  print('*' * 40)

# Modelos

Aqui, definos as duas classes de modelos: Modelo baseado em camadas lineares, que usará as representações Bag of Words e o modelo baseado na camada Embedding que usará a representação como uma sequência de índices. Um detalhe muito importante é que, para funcionar de maneira igual, é necessário realizar uma soma após a camada de embedding e, além disso, iniciar as camadas com os mesmos pesos.

In [ ]:
class LinearModel(torch.nn.Module):
  def __init__(self, input_size, emb_size):
    super(LinearModel, self).__init__()
    self.dense = torch.nn.Sequential(
        torch.nn.Linear(input_size, emb_size, bias = False),
        torch.nn.ReLU(),
        torch.nn.Linear(50,1, bias = False),
        torch.nn.Sigmoid()
    )

  def forward(self,x):
    return self.dense(x)

In [ ]:
class EmbModel(torch.nn.Module):
  def __init__(self, n_classes, emb_size, padding_idx):
    super(EmbModel, self).__init__()
    self.emb_layer = torch.nn.Embedding(n_classes, emb_size, padding_idx = padding_idx)
    self.relu = torch.nn.ReLU()
    self.linear = torch.nn.Linear(emb_size, 1, bias=False)
    self.out = torch.nn.Sigmoid()

  def forward(self,x):
    x = self.emb_layer(x)
    x = torch.sum(x,dim = 1)
    x = self.relu(x)
    x = self.linear(x)
    x = self.out(x)
    return x

# Experimentos

## Hiperparâmetros

In [ ]:
max_size = None
batch_size = 64
n_epochs = 10
learningRate = 0.01
emb_size = 50

## Bag of Words

In [ ]:
set_seed()
save_filename = 'bow'
vectorizer = BagOfWords(vocab=VOCAB)

vectorized_train_bow, vectorized_valid_bow, vectorized_test_bow, _ = create_nn_input(tokenized_x_train, y_train,
                                                                         tokenized_x_valid, y_valid,
                                                                         tokenized_x_test, y_test,
                                                                         vectorizer, batch_size)
bow_model = LinearModel(len(VOCAB), emb_size)
initial_weights = [bow_model.dense[0].weight.clone(),
                   bow_model.dense[2].weight.clone()]

bow_model.to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(bow_model.parameters(), lr=learningRate)
set_seed()
_, train_losses_bow, valid_losses_bow = train(bow_model, vectorized_train_bow, vectorized_valid_bow, criterion,
          optimizer, save_filename, n_epochs=n_epochs)

Época: 0/9 Train Loss: 0.010182 Valid Loss: 0.009688 Valid Acc: 0.695
best model
Época: 1/9 Train Loss: 0.008973 Valid Loss: 0.008618 Valid Acc: 0.746
best model
Época: 2/9 Train Loss: 0.007986 Valid Loss: 0.007795 Valid Acc: 0.786
best model
Época: 3/9 Train Loss: 0.007250 Valid Loss: 0.007226 Valid Acc: 0.806
best model
Época: 4/9 Train Loss: 0.006721 Valid Loss: 0.006831 Valid Acc: 0.814
best model
Época: 5/9 Train Loss: 0.006325 Valid Loss: 0.006543 Valid Acc: 0.822
best model
Época: 6/9 Train Loss: 0.006011 Valid Loss: 0.006318 Valid Acc: 0.831
best model
Época: 7/9 Train Loss: 0.005749 Valid Loss: 0.006137 Valid Acc: 0.839
best model
Época: 8/9 Train Loss: 0.005526 Valid Loss: 0.005987 Valid Acc: 0.843
best model
Época: 9/9 Train Loss: 0.005331 Valid Loss: 0.005861 Valid Acc: 0.848
best model


In [ ]:
predict(bow_model, save_filename, vectorized_test_bow)

****************************************
Acurácia de 83.384 %
****************************************


## Embedding

In [ ]:
set_seed()
save_filename = 'emb'
vectorizer = EmbeddingVectorizer(vocab=VOCAB)

vectorized_train, vectorized_valid, vectorized_test, _ = create_nn_input(tokenized_x_train, y_train,
                                                                         tokenized_x_valid, y_valid,
                                                                         tokenized_x_test, y_test,
                                                                         vectorizer, batch_size)

emb_init_weights = OrderedDict([
                                ('emb_layer.weight', torch.cat((initial_weights[0].T, torch.zeros(1, emb_size)))),
                                ('linear.weight', initial_weights[1])
                                ])
emb_model = EmbModel(len(VOCAB)+1, emb_size, padding_idx = len(VOCAB))
emb_model.load_state_dict(emb_init_weights)
emb_model.to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(emb_model.parameters(), lr=learningRate)
set_seed()
_, train_losses_emb, valid_losses_emb = train(emb_model, vectorized_train, vectorized_valid, criterion,
          optimizer, save_filename, n_epochs=n_epochs)

Época: 0/9 Train Loss: 0.010182 Valid Loss: 0.009688 Valid Acc: 0.695
best model
Época: 1/9 Train Loss: 0.008973 Valid Loss: 0.008618 Valid Acc: 0.746
best model
Época: 2/9 Train Loss: 0.007986 Valid Loss: 0.007795 Valid Acc: 0.786
best model
Época: 3/9 Train Loss: 0.007250 Valid Loss: 0.007226 Valid Acc: 0.806
best model
Época: 4/9 Train Loss: 0.006721 Valid Loss: 0.006831 Valid Acc: 0.814
best model
Época: 5/9 Train Loss: 0.006325 Valid Loss: 0.006543 Valid Acc: 0.822
best model
Época: 6/9 Train Loss: 0.006011 Valid Loss: 0.006318 Valid Acc: 0.831
best model
Época: 7/9 Train Loss: 0.005749 Valid Loss: 0.006137 Valid Acc: 0.839
best model
Época: 8/9 Train Loss: 0.005526 Valid Loss: 0.005987 Valid Acc: 0.843
best model
Época: 9/9 Train Loss: 0.005331 Valid Loss: 0.005861 Valid Acc: 0.848
best model


In [ ]:
predict(emb_model, save_filename, vectorized_test)

****************************************
Acurácia de 83.384 %
****************************************


In [ ]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 7 operations to synchronize with Neptune. Do not kill this process.


All 7 operations synced, thanks for waiting!


## Comparação dos resultados

In [ ]:
assert torch.allclose(torch.tensor(train_losses_emb), torch.tensor(train_losses_bow))
assert torch.allclose(torch.tensor(valid_losses_emb), torch.tensor(valid_losses_bow))

# Breve explicação do motivo dos resultados terem sido iguais

Para explicar, mostrarei que utilizar um sequência de índices, que são transformados cada um em um vetor e depois somados, é o mesmo que utilizar uma representação Bag of Words baseada em contagem.

In [ ]:
vectorized_valid_bow.dataset[0][0], vectorized_valid_bow.dataset[0][0].shape #Primeiro texto, ja tokenizado (bow), do conjunto de validacao

(tensor([8., 7., 2.,  ..., 0., 0., 0.]), torch.Size([57472]))

In [ ]:
vectorized_valid.dataset[0][0] #Primeiro texto, ja tokenizado (embedding), do conjunto de validacao

tensor([  976,     1,   959,    25,    38,    72,   386,   291,   649,     6,
           98,    25,    10,    14,   267,    23,   101,   352,    17,     7,
          267,    23,    11,    77,   128,    79,   747,     0,   277,    57,
         1873,    16,   556, 10718,  2206,   145,    27,  2533,    56,    24,
            0,  4603,   864,     2,   138,    10,    13,   145,    27,    63,
         2064,    91,     7,   327,     3, 16215,     0,  2731,    16,   608,
           82,  1409,   654,    75,    53,    23,    50,   509,   145,    27,
           80,  4022,  4689,    44, 19399,     4,    27,    56,  4090,     4,
            0,  5781,     3,     0,   114,   277,   107,   370,   200,     6,
           13,  1437,    47,     0,   512,   158,    13,     1,   303,     3,
          685,  3638,   339,   295,  9951,    25,    11,     7,    12,   342,
            4,    30,     1,   303,     3,    11,    14,    17,     4,    95,
          129,    14,    33,   200,   176,    39,    11,     6, 

In [ ]:
from torch.nn import functional

onehot = functional.one_hot(vectorized_valid.dataset[0][0], num_classes=len(VOCAB) + 1) # Matriz one-hot
onehot

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
onehot.sum(dim=0), onehot.sum(dim=0).shape # temos um elemento a mais, que eh o padding

(tensor([8, 7, 2,  ..., 0, 0, 0]), torch.Size([57473]))

In [ ]:
assert torch.all(onehot.sum(dim=0)[:-1] == vectorized_valid_bow.dataset[0][0])

Assim, vemos que ao passar pela camada de embedding e garantindo que temos os mesmos pesos, ao somar a matriz resultando, teremos o mesmo resultado da saída da primeira camada linear do modelo que utilizamos Bag of Words, pois ao somar dois tokens iguais na camada linear, é o mesmo que multiplicar os pesos da camada linear por dois, e somar todos os vetores dos tokens após a camada de embedding, é o mesmo que somar todos os _pesos * contagens_ na camada linear.